# Explicando los embeddings

¿Qué son los embeddings? ¿Cómo generarlos? ¿Qué hacer con ellos?

In [1]:
%pip install transformers torch torchvision sentence-transformers astrapy python-dotenv pandas ipywidgets -q

Note: you may need to restart the kernel to use updated packages.


El paquete "transformers" provee miles de modelos pre-entrenados para realizar tareas de diferentes maodalidades como lo son texto, vision y audio. 

PyTorch es un paquete de Python que provee dos capacidades de alto nivel: 
* Cálculos con Tensores con aceleración usando GPU
* Redes neuroanles "profundas" (deep neural networks) construidas sobre un sistema tape-based autograd

El paquete torckvision consiste en datasets populares, arquitecturas modelo y transformaciones de imagen para computación visual. 

"Sentence Transformers" (llamado también SBERT) es un módulo de Python para usar y entrenar modelos de embedding novedosos. Puede ser usado para calcular embeddings usando modelos de Sentence Transformer or para calcular puntuaciones de similitud usando modelos Cross-encoder. 

Astrapy es el cliente de Python para usar la Data API de DataStax Astra. 

## Todo comienza con el tokenizado de una frase

In [ ]:
sentence = "The world is full of kings and queens Who blind your eyes and steal your dreams It is Heaven and Hell"
# Black Sabbath - Heaven & Hell

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer_output = tokenizer.tokenize(sentence)
print(tokenizer_output)

In [ ]:
tokens_embedding = tokenizer.convert_tokens_to_ids(tokenizer_output)
print(tokens_embedding)

In [ ]:
decoded_content = tokenizer.decode(tokens_embedding)
print(decoded_content)


## Los tokens no son los embeddings

Pero esto todavía no es un emdedding. Simplemente es un relación de un token con un ID númerico.
Los modelos parten de los tokens para iniciar el proseso a través de los transformres. Cada modelo tiene su propio proceso y es aquí donde se diferencian.

# Generando embeddings

(Código basado en el modelo: https://huggingface.co/intfloat/multilingual-e5-small)

In [ ]:
# Todo comienza con la generación de tokens
from transformers import  AutoTokenizer, AutoModel
import json
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
tokens = tokenizer(sentence)
print(f"""Tokens: {len(tokens["input_ids"])}""")
tokens

In [ ]:
# Utilizando el modelo para generar el embedding
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')
batch_dict = tokenizer([sentence], max_length=512, padding=True, truncation=True, return_tensors='pt')
outputs = model(**batch_dict)
outputs

In [ ]:
outputs[0][0][0]

In [31]:
# Pooling del resultado
import torch.nn.functional as F

from torch import Tensor

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
embeddings

In [ ]:
# normalize embeddings
embeddings_norm = F.normalize(embeddings, p=2, dim=1)
embeddings_norm

In [ ]:
embeddings_norm[0]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/multilingual-e5-small')
input_texts = [
    sentence
]
embedding_st = model.encode(input_texts, normalize_embeddings=True)

In [ ]:
embedding_st 

# ¿Qué hacemos con los Embeddings?

Los embeddings son utilizados para encontrar contenido por su significado, en lugar de usar las palabras o términos específicamente. 

Vamos a ver en la práctica como generar, almacenar y encontrar similitudes.

In [44]:
# Vamos a importar los datos
import pandas as pd
from tqdm import tqdm
# df = pd.read_csv("./data/90minFootballTransferNewsNLP.csv")

# Load de dataset CSV 
df = pd.read_csv("data/ProductDataset.csv", header = 0)
#  new_df = pd.DataFrame(columns=['product_id','product_name', 'description', 'price', 'embedding'])

In [ ]:
df.head()
print(df.iloc[1]["description"])

In [ ]:
from sentence_transformers import SentenceTransformer
model_emb = SentenceTransformer('intfloat/multilingual-e5-small')
print(df.iloc[1]["description"])
emb = model_emb.encode(df.iloc[1]["description"],normalize_embeddings=True)
print(emb)

## Usar una base de datos para almacenar nuestros embeddings

La base de datos para almacenar los embeddings debe contar con la capacidad de almacenar vectores y operar con ellos, como por ejemplo realizar una búsqueda vectorial. 

DataStax cuenta con las capacidades de una base de datos de Cassandra con la funcionalidad adicional de búsqueda de vectores.
Vamos a usar una "collection" para almacenar nuestros embeddings

In [ ]:
# Creamos la conexión con DataStax Astra

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import cassio
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = "astra_db" # "astra_db"/"local"
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [ ]:
# Este es el código para crear la tabla en Astra, pero usaremos coleeciones de datos en su lugar.

from cassio.table.tables import MetadataVectorCassandraTable
table_name = "football_news_emb"
embedding_dimension = 384

v_table = MetadataVectorCassandraTable(
    session=session,
    keyspace=keyspace,
    table=table_name,
    vector_dimension=embedding_dimension,
    primary_key_type="TEXT",
)

In [ ]:
# Carga de datos

rows_to_load = 100
for index, row in tqdm(df.head(rows_to_load).iterrows(),total=len(df.head(rows_to_load))):
    v_table.put(
            row_id=f"""{row["Date"]}|{row["Title"]}""",
            body_blob=row["Content"],
            vector=model_emb.encode(row["Content"]),
            metadata={"date":row["Date"], "link": row["Link"]}
        )

## Conexión usando la Data API de Astra

In [ ]:
from astrapy import DataAPIClient
from dotenv import load_dotenv
from astrapy.db import AstraDB, AstraDBCollection

load_dotenv(override=True)

coll_name = "appliances"
embedding_dimension = 384

print("AstraDB collection...")

client = DataAPIClient()
database = client.get_database(
    os.getenv("ASTRA_DB_API_ENDPOINT"),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
)

collection = database.create_collection(coll_name, dimension=embedding_dimension)


### Carga de datos

In [ ]:
from ipywidgets import IntProgress

rows_to_load = 10

# Load to vector store
def load_to_astra(df, collection):
  #len_df = len(df)
  len_df = rows_to_load

  f = IntProgress(min=0, max=len_df) # instantiate the bar
  display(f) # display the bar
  for i in range(len_df):
    f.value += 1 # signal to increment the progress bar
    f.description = str(f.value) + "/" + str(len_df)

    product_id = df.loc[i, "product_id"]
    product_name = df.loc[i, "product_name"]
    description = df.loc[i, "description"]
    price = df.loc[i, "price"]
    if(type(price) is float):
      price = 0

    # Vector elements are numpy.float32, which is not JSON serializable, .tolist() converts to native Python float
    embedding = model_emb.encode(df.loc[i, "description"], normalize_embeddings=True).tolist() 
    
    try:
      #add to the Astra DB Vector Database
      collection.insert_one({
        "_id": str(product_id),
        "product_name": product_name,
        "description": description,
        "price": price,
        "$vector": embedding,
        })
    except Exception as error:
      #if you've already added this record, skip the error message
      if str(error) == "Document already exists with the given _id":
        print("Document already exists in the database. Skipping.")

load_to_astra(df, collection)


In [ ]:
print(collection.count_documents(filter={}, upper_bound=100))

## Vectorize

In [ ]:
vectorize_coll = "appliances_nvidia"
vect_collection = database.get_collection(vectorize_coll)

print(vect_collection.count_documents(filter={}, upper_bound=100))

In [ ]:
from ipywidgets import IntProgress

vectorize_coll = "appliances_nvidia"
vect_collection = database.get_collection(vectorize_coll)

rows_to_load = 2

# Load using vectorize
def load_with_vectorize(df, collection):
  len_df = rows_to_load #len_df = len(df)

  f = IntProgress(min=0, max=len_df) # instantiate the bar
  display(f) # display the bar
  for i in range(len_df):
    f.value += 1 # signal to increment the progress bar
    f.description = str(f.value) + "/" + str(len_df)

    product_id = df.loc[i, "product_id"]
    product_name = df.loc[i, "product_name"]
    description = df.loc[i, "description"]
    price = df.loc[i, "price"]
    if(type(price) is float):
      price = 0
    
    try:
      #add to the Astra DB Vector Database
      collection.insert_one({
        "product_id": str(product_id),
        "product_name": product_name,
        "description": description,
        "price": price,
        "$vectorize": product_name + ": " +description,
        })
    except Exception as error:
      #if you've already added this record, skip the error message
      if str(error) == "Document already exists with the given _id":
        print("Document already exists in the database. Skipping.")

load_with_vectorize(df, vect_collection)

## Búsqueda por similaridad de vectores

In [ ]:
query = "reproductores de CD y DVD"
query_emb = model_emb.encode(query)

results = collection.find(sort={"$vector": query_emb}, limit=5, include_similarity=True)
for result in results:
    print("--------------------")
    print(f"""Distance: {result["$similarity"]}""")
    print(f"{result["product_name"]} : {result["description"]}")
